In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib as mat
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Abrindo o dataset
file_name='/kaggle/input/housesalesprediction/kc_house_data.csv'
df=pd.read_csv(file_name)
df.head()

# Analisando os dados

In [ ]:
df.columns

* id :- It is the unique numeric number assigned to each house being sold.
* date :- It is the date on which the house was sold out.
* price:- It is the price of house which we have to predict so this is our target variable and aprat from it are our features.
* bedrooms :- It determines number of bedrooms in a house.
* bathrooms :- It determines number of bathrooms in a bedroom of a house.
* sqft_living :- It is the measurement variable which determines the measurement of house in square foot.
* sqft_lot : It is also the measurement variable which determines square foot of the lot.
* floors: It determines total floors means levels of house.
* waterfront : This feature determines whether a house has a view to waterfront 0 means no 1 means yes.
* view : This feature determines whether a house has been viewed or not 0 means no 1 means yes.
* condition : It determines the overall condition of a house on a scale of 1 to 5.
* grade : It determines the overall grade given to the housing unit, based on King County grading system on a scale of 1 to 11
* sqft_above : It determines square footage of house apart from basement.
* sqft_basement : It determines square footage of the basement of the house.
* yr_built : It detrmines the date of building of the house.
* yr_renovated : It detrmines year of renovation of house.
* zipcode : It determines the zipcode of the location of the house.
* lat : It determines the latitude of the location of the house.
* long : It determines the longitude of the location of the house.
* sqft_living15 : Living room area in 2015(implies-- some renovations)
* sqft_lot15 : lotSize area in 2015(implies-- some renovations)

In [ ]:
print(df.shape[0]) #Número de linhas
print(df.shape[1]) #Número de colunas

In [ ]:
df.info()

In [ ]:
#Descrição do dataset
df.describe()

In [ ]:
df.dtypes

# Limpando os dados

In [ ]:
#Valores NaN
df.isnull().sum()

#### Substituindo valores nulos das colunas bedrooms e bathrooms pela média de cada coluna

In [ ]:
med_bed = df['bedrooms'].mean()
med_bath = df['bathrooms'].mean()

#Arredondando valores
import math
med_bed = math.floor(med_bed)
med_bath = math.floor(med_bath)

print(med_bed)
print(med_bath)

In [ ]:
df['bedrooms'].fillna(med_bed, inplace=True)
df['bathrooms'].fillna(med_bath, inplace=True)


#Agora não há nenhum valor nulo no dataset
df.isnull().sum()

In [ ]:
#Não há valores duplicados no dataset
df.duplicated().sum()

In [ ]:
#Analisando os outliers. É possível ver que não existe nenhum valor absurdo que possa comprometer a análise do dataset 
plt.figure(figsize=(12,6))
df.boxplot()
plt.xticks(rotation = 90);

In [ ]:
plt.figure(figsize=(25,12))
df_corr = df.corr()
sns.heatmap(df_corr, annot = True)
print()

In [ ]:
#Salvando o dataset
df.to_csv('kc_house.csv')

# Hipóteses

* 1 - As casas que passaram por renovação recentemente possuem as melhores notas?
* 2 - O preço médio das casas subiu 2% ao mês em 1 ano?
* 3 - O número de quartos influencia no preço médio das casas?
* 4 - As casas mais bem avaliadas, são em geral, 20% mais caras do que as menos bem avaliadas?
* 5 - As casas mais bem avaliadas estão na orla da cidade?

# H01 - As casas que passaram por renovação recentemente possuem as melhores notas?

* Falso. Casas que passaram por reformas mais antigas têm boas notas, assim como as que passaram por reformas recentemente.

In [ ]:
from datetime import datetime
now = datetime.now()
year = now.year
year

In [ ]:
df['yr_dif'] = year - df['yr_renovated']
hx1 = df[['yr_dif','grade']]
hx1

In [ ]:
hx1 = hx1[hx1['yr_dif']!= 2020]
plt.figure(figsize=(18,12))
plt.title('Média de nota por reforma', fontsize=33)
plt.ylabel('',fontsize=22)
plt.xlabel('',fontsize=22)
sns.barplot(x='yr_dif', y='grade',data=hx1)
plt.xticks(rotation = 90);

# H02 - O preço médio das casas subiu 2% ao mês em 1 ano?

* Falso. O preço médio das casas variou de mês para mês no período de 1 ano

In [ ]:
#Eliminei toda a bagunça que estava na coluna date e peguei apenas os dados que me interessavam: mês e ano.
df['date']=[x[:6] for x in df['date']]
df.head()

In [ ]:
plt.figure(figsize=(15,12))
plt.title('Média de preço em 1 ano', fontsize=33)
plt.ylabel('',fontsize=22)
plt.xlabel('',fontsize=22)
h02 = df[['date','price']].groupby('date').mean().reset_index()
h02['growth_price'] = 100*h02['price'].pct_change()
sns.barplot(x='date', y='growth_price',data=h02)
plt.xticks(rotation = 45);

In [ ]:
h02['growth_price']

# H03 - O número de quartos influencia no preço médio das casas?

* Falso. Há casas com 11 quartos que são mais baratas do que casas com 4 quartos, por exemplo.

In [ ]:
#Há uma casa com 33 quartos. Poder ser um outlier e por isso, foi retirado.
df=df[df['bedrooms']!=33]
h03 = df[['bedrooms','price']].groupby('bedrooms').mean().reset_index()

In [ ]:
plt.figure(figsize=(15,10))
plt.title('Média de preço por número de quartos', fontsize=33)
plt.ylabel('',fontsize=22)
plt.xlabel('',fontsize=22)
sns.barplot(x='bedrooms', y='price',data=h03);

# H04 - As casas mais bem avaliadas, são em geral, 20% mais caras do que as menos bem avaliadas?

* Falso. Casas com avaliação 3 são mais caras do que casas com avaliação 4, assim como casas com avaliação 9 são mais caras que casa com avaliação 10.

In [ ]:
plt.figure(figsize=(15,8))
plt.title('Média de preço por avaliação', fontsize=33)
plt.ylabel('',fontsize=22)
plt.xlabel('',fontsize=22)
h04 = df[['grade', 'price']].groupby('grade').mean().reset_index()
h04['growth'] = 100*h04['price'].pct_change()
sns.barplot(x='grade', y='growth',data=h04);

In [ ]:
print(h04)

# H05 - As casas mais bem avaliadas estão na orla da cidade ?

* Verdadeiro. A região com o zipcode 98039, no distrito de Medina, na baía de Seattle, possui uma avaliação média de 9.56 de 11 e fica à beira do Lago Washington.



In [ ]:
plt.figure(figsize=(17,12))
plt.title('Média de avaliação por região', fontsize=33)
h05 = df[['zipcode', 'grade']].groupby('zipcode').mean().reset_index() #Criando a tabela
plot_order = h05.sort_values(by='grade', ascending=False).zipcode      #Ordenando a tabela
sns.barplot(x='zipcode', y='grade',data=h05, order=plot_order)         #Plotando a tabela
plt.ylabel('',fontsize=22)
plt.xlabel('',fontsize=22)
plt.xticks(rotation = 90);

In [ ]:
h05.sort_values(by=['grade'],ascending=False).head()